## Strategy Pattern

## Inheritance and Composition

Use David's example: A is a B vs A is made up of B.

## Dependency Inverson Principle

### Definition

In object-oriented design, the [dependency inversion principle](https://en.wikipedia.org/wiki/Dependency_inversion_principle) is a specific methodology for loosely coupling software modules. When following this principle, the conventional dependency relationships established from high-level, policy-setting modules to low-level, dependency modules are reversed, thus rendering high-level modules independent of the low-level module implementation details. The principle states:[1]

- High-level modules should not import anything from low-level modules. Both should depend on abstractions (e.g., interfaces).
- Abstractions should not depend on details. Details (concrete implementations) should depend on abstractions.
  
By dictating that both high-level and low-level objects must depend on the same abstraction, this design principle inverts the way some people may think about object-oriented programming.[2]

References:

1. [Abstractions should not depend on implementations.](https://stackoverflow.com/questions/52857145/what-is-mean-by-abstractions-should-not-depend-on-details-details-should-depen)
2. [Wikipedia](https://en.wikipedia.org/wiki/Dependency_inversion_principle)

### Low Level and High Level Modules

Low level modules are "low level" because they have no dependencies, or no relevant dependencies. Very often, they can be easily reused in different contexts without introducing any separate, formal interfaces - which means, reusing them is straightforward, simple and does not require any Dependency Inversion.

High level modules, however, are "high level", because they require other, lower level modules to work. But if they are tied to a specific low-level implementation, this often prevents to reuse them in a different context.

High level modules depend on low level modules, but shouldn't depend on their implementation. This can be achieved by using interfaces, thus decoupling the definition of the service from the implementation.

References: 

1. https://stackoverflow.com/questions/3780388/what-are-high-level-modules-and-low-level-modules-in-the-context-of-depende

### Example

- `CustomDataset` is a high-level module that will apply transformation to the data and return
the transformed data depending on the `stage`.
- `ImageClassificationTransforms` is a low-level module that will return the transformation functions.
- In our code, the high level module depends on the low level module such that the **creation** of 
`ImageClassificationTransforms` is done inside the `CustomDataset` constructor. This leads to **high coupling**. 
- The code looks fine but if we want to change the `ImageClassificationTransforms` to `ImageSegmentationTransforms`, then we have to change the `CustomDataset` code in two places:
    - Type hint of `ImageClassificationTransforms` to `ImageSegmentationTransforms`;
    - Change manually the `ImageClassificationTransforms` to `ImageSegmentationTransforms` in the constructor.
- Things soon get out of hand if we have a lot more of such dependencies, such as `ObjectDetectionTransforms`, `ImageCaptioningTransforms`, etc.


- One way to solve this is to use the **Dependency Inversion Principle**. We can create an interface `Transforms` that will be implemented by `ImageClassificationTransforms`, `ImageSegmentationTransforms`, etc. Then, we can pass the `Transforms` object to the `CustomDataset` constructor `__init__` method. This way, the `CustomDataset` will depend on the `Transforms` interface and not on the `ImageClassificationTransforms` class. This way, we can change the `ImageClassificationTransforms` to `ImageSegmentationTransforms` without changing the `CustomDataset` code. This is called **Dependency Inversion**.

The abstraction does not depend on details simply mean the abstract class should not
hold any implementation. The implementation should be done in the concrete class.

Example:

In my `Transforms(ABC)` abstract class/interface below, I have two abstract methods `get_train_transforms` and `get_test_transforms`. These methods are not implemented in the abstract class. They are implemented in the concrete class `ImageClassificationTransforms`. This is the second rule in **Dependency Inversion Principle**.

In the high level module `CustomDataset`, I have a constructor `__init__` that takes in a `Transforms` abstract class/interface. Now my `CustomDataset` depends on the `Transforms` abstraction and not on the `ImageClassificationTransforms` class. This is the first rule in **Dependency Inversion Principle**. Furthermore, if you were to switch your task
from image classification to image segmentation, you can simply change the `ImageClassificationTransforms` to `ImageSegmentationTransforms` without changing the `CustomDataset` code as you are not **creating/coupled** to the `ImageClassificationTransforms` class.

```python
class Transforms(ABC):
    """Abstract class for transforms."""

    @abstractmethod
    def get_train_transforms(self) -> Callable:
        """Get train transforms."""


    @abstractmethod
    def get_test_transforms(self) -> Callable:
        """Get test transforms."""

class ImageClassificationTransforms(Transforms):
    """Dummy class for image classification transforms."""

    def get_train_transforms(self) -> Callable:
        """Get train transforms."""
        print("Getting image classification train transforms.")
        return lambda x: None

    def get_test_transforms(self) -> Callable:
        """Get test transforms."""
        print("Getting image classification test transforms.")
        return lambda x: None


class ImageSegmentationTransforms(Transforms):
    """Dummy class for image segmentation transforms."""

    def get_train_transforms(self) -> Callable:
        """Get train transforms."""
        print("Getting image segmentation train transforms.")
        return lambda x: None

    def get_test_transforms(self) -> Callable:
        """Get test transforms."""
        print("Getting image segmentation test transforms.")
        return lambda x: None


class CustomDataset:
    def __init__(self, transforms: Transforms, stage: str = "train") -> None:
        self.stage = stage
        self.transforms = transforms

    def apply_transforms(self, dummy_data: Any = None) -> Any:
        """Apply transforms to dataset based on stage."""
        if self.stage == "train":
            transformed = self.transforms.get_train_transforms()(dummy_data)
        else:
            transformed = self.transforms.get_test_transforms()(dummy_data)
        return transformed
```


Originally, `CustomDataset` **creates** its own dependency and it is the one controlling
the dependency. Now after applying **Dependency Inversion Principle**, `CustomDataset` is no longer creating its own dependency. It is now **injected** with the dependency. This **inverts
the control** of the dependency from `CustomDataset` to the caller of `CustomDataset`. This is the **Dependency Inversion Principle**.

More concretely, in traditional sense, since class A depends on class B, then class A is the one creating the dependency. But after applying **Dependency Inversion Principle**, class A is no longer creating the dependency. Instead, the dependency is instantiated outside of class A 
at runtime and is injected into class A. This is the **Dependency Inversion Principle**,
a form of **Inversion of Control**.

In [1]:
# convert C# to Python
class AppPoolWatcher: #  high level module
    def __init__(self):
        self.writer = EventLogWriter()

    def notify(self, message):
        self.writer.write(message)
        
        
class EventLogWriter: # low level module
    def write(self, message):
        # Write to event log here
        pass

High level module `AppPoolWatcher` depends on low level module `EventLogWriter`
where `EventLogWriter` is a concrete class and not an interface/abstract class.
creation of `EventLogWriter` is tightly coupled with `AppPoolWatcher` class.
in other words, `AppPoolWatcher` creates `EventLogWriter` object since
we are creating `EventLogWriter` object inside the constructor of `AppPoolWatcher`.

Now the problem is that if we want to change the implementation of `EventLogWriter`
then we will have to change the `AppPoolWatcher` class as well. This is not a good
design. For example, if we want our `AppPoolWatcher` to write the error messages
to an email instead of event log then we will have to change the `self.writer` to
say, `EmailWriter` and then create an instance of `EmailWriter` inside the class.


The problem will get even worse when we have more actions to take selectively, like sending SMS. 
Then we will have to have one more class whose instance will be kept inside the AppPoolWatcher. 
The dependency inversion principle says that we need to decouple this system in such 
a way that the higher level modules i.e. the `AppPoolWatcher` in our case will depend
on a simple abstraction and will use it. This abstraction will in turn will be 
mapped to some concrete class which will perform the actual operation. (Next we will see how this can be done)  

## Dependency Injection

- https://stackoverflow.com/questions/1314730/which-design-patterns-can-be-applied-to-the-configuration-settings-problem
- https://www.youtube.com/watch?v=2ejbLVkCndI
- http://martinfowler.com/articles/injection.html

## Terminology

- Polymorphism: 

## Further Readings

- https://eugeneyan.com/writing/design-patterns/
- https://github.com/msaroufim/ml-design-patterns
- https://refactoring.guru/design-patterns
- https://en.wikipedia.org/wiki/Design_Patterns